# Lunar Surface

© Dimitrios Theodorakis GNU General Public License v3.0 https://github.com/DimitriosAstro/Astronomy

In [ ]:
import numpy as np
from scipy import ndimage
from skimage import io
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import Image
from skyfield.api import PlanetaryConstants, load
%matplotlib notebook

## AIM - To compare our moon image to known data and work out some feature sizes

**Predict**

Have a think about these questions and make some predictions! Be sure to tell someone else what you predict.

1) Do the near and far sides of the moon look the same?  
2) What sort of surface features do you think you can find on the moon?  
3) How do we measure the height of the moon's surface?

# Let's go:

In this notebook you will first compare your image of the moon with our telescope with a map from the Lunar and Planetary Institute (LPI). Upload your image by clicking on ***File - Open*** which opens the dashboard view then the ***upload*** button on the top right. Rename it to **mymoon.jpg** (or png) and load it below:

In [ ]:
moon = io.imread('mymoon.jpg')

In [ ]:
# Your image will need rotating since the telescope flips the image
# you might have to change this angle from 225 degrees!
moon_rot = ndimage.rotate(moon, 225)

In [ ]:
# Make a nice big figure
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(moon_rot, aspect='equal')
ax.axis('off')

You should see your image of the moon! Now you need to get the map (or maps) of the moon for comparison from the LPI.

Navigate to: https://www.lpi.usra.edu/resources/mapcatalog/LMP/ and download a map. I suggest you start with the Earthside map of the moon (this file is also stored in this directory as **lunarnear.jpg**). This map shows the side of the moon that we see but not at high or low latitudes. If you want to identify features at high or low latitudes I suggest you also download the Polar map.

Identify at least 10 features in both your map and the LPI maps. You're going to annotate your image with these features.  
```ax.annotate()``` will draw an annotation with an arrow  
```ax.text()``` will just draw text on your image

Example code for both is shown below. I suggest for features such as mare you use ```ax.text()``` and for smaller features you use ```ax.annotate()```. Get the pixel coordinates for a feature by hovering over the figure. The x, y location should appear in the lower right hand corner.

In [ ]:
# Re-running this code multiple times will overplot annotations 
# run the last cell again before this one to avoid overplotting

# Example annotation make sure there is space for the arrow
ax.annotate('Manilius', xy=(1300, 1312), xycoords='data', xytext=(900,1200), textcoords='data',
            color='white', arrowprops=dict(arrowstyle='->', color='white'))
# Example text (no arrow) - here \n means new line
ax.text(1480, 1060, 'Mare \n Serenitatis', color='k', ha='center')

# Your code here:


When you're happy with the annotated figure save it:

In [ ]:
fig.savefig('mymoonAnnotated.png', dpi=200, bbox_inches='tight')

Great! But there must be more data we can extract from our image right?

We can use information about our telescope and camera to work out the size of the features you labelled. Do do this we need to know a few things:

 - The size of the pixels on our camera in microns
 - The focal length of our telescope in mm
 
You'll have to look up both these values online, perhaps google: Camera/Phone Model **Sensor** and then google the sensor. If you can't find it at all then a good approximation might be $1.3\mu m$. If you used a Canon, Nikon, or professional CCD you can go to https://astronomy.tools/calculators/ccd_suitability and enter your camera there and it will display the pixel size for you.
From these two pieces of information we can calculate the resolution of our camera:

$$Resolution\ [arc\ seconds\ per\ pixel] = \frac{Pixel\ size\ [\mu m]}{Focal\ length\ [mm]}\times206.265$$

Use this formula to calculate the resolution of our camera:

In [ ]:
# Your code here:
pixelSize =  # micro m
fLength =  # mm
const = 206.265
# If you used a smartphone multiply your resolution by the focal length of your smartphone camera
fLengthSmartphone = 1

res = 
res

Now you have the resolution of your camera and telescope setup we can calculate the diameter of a crater. Locate a crater near the terminus of the moon and measure the width in pixels of the crater by hovering over the image:

In [ ]:
# Your code here:
dx = 
dx

We can use trigonometry to calculate the width of the crater. The angle, ```theta```, in our triangle is the pixel width, ```dx```, times our resolution, ```res```, converted to radians and the adjacent side is the distance to that lunar feature, ```dist```. The opposite is then the width of the crater. Using our trig functions we find ```width = np.tan(theta)*dist```.

We can either use the average distance to the moon for the ```dist``` or better yet we can use a package called **SkyField** which will claculate the distance to that lunar feature.

Adapt the SkyField code below to calculate the distance to the lunar crater you want to measure the height of:

In [ ]:
# Your code here:
ts = load.timescale(builtin=True)
# Change this to the time of your observation - year, month, day, hour, min
t = ts.utc(2019, 12, 3, 17, 32)

eph = load('de421.bsp')
earth, moonSky = eph['earth'], eph['moon']

pc = PlanetaryConstants()
pc.read_text(load('moon_080317.tf'))
pc.read_text(load('pck00008.tpc'))
pc.read_binary(load('moon_pa_de421_1900-2050.bpc'))

frame = pc.build_frame_named('MOON_ME_DE421')
# Change the name of the next variable to the name of the crater you chose
# look up its latitude and longitude and enter them on the line below
# remember South lats and W longs are entered as negative numbers
theophilus = moonSky + pc.build_latlon_degrees(frame, 26.4, -11.4)

# Change the crater name in the next line
apparent = earth.at(t).observe(theophilus).apparent()
ra, dec, distance = apparent.radec(epoch='date')
distance

We have the distance to our feature! Let's convert it to km:

In [ ]:
dist = distance.km
dist

Now calculate the width of the crater:

In [ ]:
# Your code here:
theta =  # *np.pi/(180*3600) converts arcseconds to radians
width = 
# Change the longitude in the correction to the longitude of your crater (ignore minus signs)
correction = np.cos(11.4*np.pi/180)
width = width/correction
width

Does this value match up with the accepted value online? If not why? You should have gotten pretty close.

You'll notice I added a correction term based on the crater's longitude. This takes into account the fact that we are seeing the crater at an angle and not face on unless it lies exactly at a longitude of zero.

Be sure to add this value to your annotations (don't re-save your figure yet we'll do that later).

In [ ]:
text1 = 'Theophilus\nMy width = '+str(round(width))+' km \nActual width = 100 km'
ax.annotate(text1, xy=(1614, 1725), xycoords='data', xytext=(760,1680), textcoords='data',
            color='white', arrowprops=dict(arrowstyle='->', color='white'), ha='center')

Now you need to choose another crater close to the terminus where you can clearly see a shadow in the crater. We are going to use the shadow to calculate the depth of the crater. First let's calculate the width of the crater like before:

In [ ]:
# Your code here:
dx2 = 

# Change the name of the next variable to the name of the crater you chose
# look up its latitude and longitude and enter them on the line below
# remember South lats and W longs are entered as negative numbers
abulfeda = moonSky + pc.build_latlon_degrees(frame, -13.8, 13.9)

# Change the crater name in the next line
apparent2 = earth.at(t).observe(abulfeda).apparent()
ra2, dec2, distance2 = apparent2.radec(epoch='date')

dist2 = distance2.km

theta2 =  # *np.pi/(180*3600) converts arcseconds to radians
width2 = 
# Change the longitude in the correction to the longitude of your crater (ignore minus signs)
correction2 = np.cos(13.9*np.pi/180)
width2 = width2/correction2
width2

So we have our width like before but how do we calculate the depth?

Look at this figure from the University of Iowa (found [here](http://astro.physics.uiowa.edu/ITU/labs/observational-labs/studying-the-moon/measure-the-height-of-lunar/)):

In [ ]:
Image('lunarcrater_med.png')

Pause and come up with the trig relationship between $H$, $L$, and $\theta$.

Now look at this second figure also from the University of Iowa:

In [ ]:
Image('earthmoonsuntri_med.png')

It shows the sun illuminating a portion of the moon and our view from Earth. Here $R$ is the radius of the moon and $d$ is the distance from the terminator to the crater. Take a moment to get a trig relationship between $R$, $d$, and $\theta$.

If we use the small angle approximation:

$$tan\theta = sin\theta = \theta$$

which holds for angles less than $\approx 10 \deg$ we can combine these two trig identities to get the formula for the height of the crater:

$$H\approxeq L\times\frac{d}{R}$$

we will take $R = 1737.1\ km$. 

To calculate L work out the width in pixels of the crater by hovering over the image. Divide that by the width in pixels of the crater. This gives you the fraction of the total width of the crater in km that the shadow covers:

In [ ]:
# Your code here:
dxShadow = 
ratio = 
L = 
L

Now measure $d$ in pixels from your image. Remember d is the distance from the terminator to the center of the crater.

In [ ]:
# Your code here:
d = 
d

Convert this $d$ into an angle using the resolution from earlier and then turn that angle into a distance:

In [ ]:
# Your code here:
theta_d = 
d_km = 

All that's left is to calculate $H$:

In [ ]:
# Your code here:
H = 
H

How does your value compare to the actual value?

In our code we just used dx values but the get more accurate distances we should have calculated the distance between two x, y points using trig. If you have time go back and make these changes. How do your values change?

Let's now add this new crater info onto your image and save it one last time:

In [ ]:
text2 = 'Abulfeda\nMy width = '+str(round(width2))+' km\nActual width = 62 km\nMy depth = '+str(round(H, 1))+' km\nActual depth = 3.1 km'
ax.annotate(text2, xy=(1410, 1796), xycoords='data', xytext=(770, 2170), textcoords='data',
            color='white', arrowprops=dict(arrowstyle='->', color='white'), ha='center')

Let's add a simple title and save the figure for the last time:

In [ ]:
ax.text(1330, 188, 'Dimitrios Moon Obs\n2019-12-03 17:32', color='w', ha='center')
fig.savefig('mymoonAnnotated.png', dpi=200, bbox_inches='tight')

# LRO and LOLA

When you've been looking up some of the features/craters for your image you will have noticed some very nice pictures. Some of these are from the Lunar Reconnaissance Orbiter ([LRO](http://lroc.sese.asu.edu/)) and the Lunar Orbiter Laser Altimeter ([LOLA](https://lola.gsfc.nasa.gov/)).

Explore the LRO site for images of features you labelled to compare to your image. I suggest you start by heading to their Archive tab then the Lunaserv Global Explorer which will let you explore all of the moons surface. Zoom in and take some screenshots of the features you labelled. You can also explore LRO data using the JMARS tool [here](https://jmars.mars.asu.edu/).

When you've found an image or screenshotted one of a crater you previously labelled upload it as a jpeg to Jupyter (File - Open will take you to the dashboard then use the Upload button). We're going to plot this image next to an enlarged section of your image centered on that crater: 

In [ ]:
fig2 = plt.figure(num=2, figsize=(8,8))
ax1 = plt.subplot2grid((2,2), (0,0), rowspan=1, colspan=2)
ax2 = plt.subplot2grid((2,2), (1,0), rowspan=1)
ax3 = plt.subplot2grid((2,2), (1,1), rowspan=1)
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

# The top of this graph will show our moon image unrotated
ax1.imshow(moon, aspect='equal')

# # The next image we will crop our image to 'zoom into' the crater
# # run the code with these lines commented and hover over the top image to get the x, y points
# # enter the numbers in the line below, ymin:ymax, xmin,xmax
# moon_cropped = moon[610:680, 990:1080, :]
# # Rotate your image so it lines up with the LRO image
# moon_rot_crop = ndimage.rotate(moon_cropped, 190)
# ax2.imshow(moon_rot_crop, aspect='equal')

# Load and plot the LRO image you downloaded/screenshotted in the bottom right
lro = io.imread('theophilusLROC.jpg')
ax3.imshow(lro, aspect='equal')

# Change the title!
Title = 'Dimitrios Full Moon Obs 2019-12-03 17:32\nBottom Left: Theophilus Crater Enlarged\nBottom Right: Theophilus Crater LROC'
plt.suptitle(Title)
plt.subplots_adjust(wspace=0.1, hspace=0.1)

When you're happy with this figure save it:

In [ ]:
fig2.savefig('theophilusCraterComparison.png', dpi=200)

Where does LOLA fit in you ask? LOLA measures the height of the lunar surface using lasers. You can see the products of the LOLA satellite on their website. The data from LOLA is used to make digital elevation models (DEM) of the lunar surface at a very high resolution.

I have included some LOLA data for you to look at if you want. The LOLA DEM in this directory is stored as an IMG file. Open it up with a text editor to examine the header. It has lots of useful information like the resolution.

The program Generic Mapping Tools ([GMT](https://www.generic-mapping-tools.org/)) can be used to convert this IMG data file to a standard image format for viewing:

In [ ]:
# the ! infront of the command lets Jupyter know we want to run this in the command line
!gmt xyz2grd -ZTLf LDEM_16_FLOAT.IMG -GLDEM_16_FLOAT.GRD -R0/360/-90/90 -I0.0625 -r

Now let's load the result:

In [ ]:
Image('Lunar_LROC_WAC_GLD100_ClrShade_79s79n_9216.jpg')

Wow! This is a cylindrical projection of the entire lunar surface with blue areas lower than black (high) areas.

If you want to look at more of the data products from both LRO and LOLA head on over to the Lunar Orbiter section of the [PDS Geoscience Node](https://ode.rsl.wustl.edu/moon/index.aspx) at Washington University.

I have included some raw LOLA data as csv's for you to have a play with in pandas and matplotlib in the subdirectory **./RDR_272E273E_46p130529S45SPointPerRow_csv_table**. This data is centered on the Catalan crater. Each .csv file has an associated .lbl file with information about the .csv file (like a header).

This animation of the Catalan crater was made with the **RDR_272E273E_46p130529S45STopoFull_csv_table.csv** file and matplotlib's 3D plotting functionality (the code is stored in **catalan.py**).

In [ ]:
Image('catalan.gif')

# Challenge

1) Change the code we used to calculate width and depths of craters to use x,y points to calculate distance as described above  
2) Have a go at processing the included LOLA data on the Catalan crater  
3) Change the code to use the method described in the University of Exeter reference to calculate depths which doesn't rely on the small angle approximation  

Here is a code snippet you could add after the first calls to SkyField to get the [sub Earth](https://rhodesmill.org/skyfield/planetary.html#computing-lunar-libration) and sub Solar points on the moon for the Exeter method. I have added an ***exeter.py*** file with this code snippet and the lines after which calculate the height with this method (you get more accurate heights!).

```
sun = eph['sun']

p = (earth - moonSky).at(t)
latSubE, lonSubE, distanceSubE = p.frame_latlon(frame)
lonSubE_degrees = (lonSubE.degrees - 180.0) % 360.0 - 180.0
print('sub-Earth point latitude: {:.3f}'.format(latSubE.degrees))
print('sub-Earth point longitude: {:.3f}'.format(lonSubE_degrees))

p2 = (sun - moonSky).at(t)
latSubS, lonSubS, distanceSubS = p2.frame_latlon(frame)
lonSubS_degrees = (lonSubS.degrees - 180.0) % 360.0 - 180.0
print('sub-Solar point latitude: {:.3f}'.format(latSubS.degrees))
print('sub-Solar point longitude: {:.3f}'.format(lonSubS_degrees))
```

4) HARD - Have a go at looking at or processing LRO/LOLA data with GMT and the PDS Geoscience Node

# References

Resolution of ccd/telescope, Astronomy Tools, https://astronomy.tools/calculators/ccd_suitability  
Depth of Craters - Univeristy of Iowa, http://astro.physics.uiowa.edu/ITU/labs/observational-labs/studying-the-moon/measure-the-height-of-lunar/  
Crater Diameters - University of Exeter, http://www.astro.ex.ac.uk/obs/experiments/lunar/script.html  

Lunar Orbital Data Explorer, PDS Geosciences Node - Washington University, https://ode.rsl.wustl.edu/moon/index.aspx    
JMARS Tool, https://jmars.mars.asu.edu/  
Lunar Reconnaissance Orbiter, http://lroc.sese.asu.edu/  
Lunar Orbiter Laser Altimeter, https://lola.gsfc.nasa.gov/

# Acknowledgements

Thanks to [Dr. John Keller](https://science.gsfc.nasa.gov/sed/bio/john.w.keller) for his guidance towards the GMT tool and processing of LOLA .IMG data files.

Thanks also to Brandon Rhodes author of the [SkyField](https://rhodesmill.org/skyfield/) package for advice on calculating sub-Earth and sub-Solar points on the moon.